In [5]:
# Import necessary modules for scraping
from bs4 import BeautifulSoup as bs
import pandas as pd
# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


## NFL Data Scraping
* Dynamic (Franchise Value 2023) https://www.statista.com/statistics/193534/franchise-value-of-national-football-league-teams-in-2010/
    
    1. click on "adjust statistic presentation button (resembles normal "settings" button), button element:
    ```
    <button aria-label="Settings" aria-haspopup="true" data-gtm="statisticGraph__icon--settings" type="button" class="toolbar__item panelCard padding-all-10 flex-s--grow" tabindex="0"><svg aria-hidden="true" height="12" width="18" class="toolbar__icon uiIcon--default"><use href="#iconAdjustmentFill"></use></svg></button>
    ```
    2. select grid option, element:
    ``` 
    <button aria-label="Display table" class="button button--active"><svg aria-hidden="true" height="16" width="16"><use href="#iconChartTableFill"></use></svg></button>
    ```
    3. Select 50 under Records drop down
    ``` 
    <select id="dataTablePageSize1717123935205"><option> <option>50</option> 
    ```
    4. data is now present in "Inspect"
    5. data is in this format in HTML 
    ```
    <span>New York Giants</span> , <span>6.8</span>```
     ```
     <tr> </tr>
     ``` 
    in table row format
    

In [9]:
# 1. Dynamic (Franchise Value 2023)
# Set up WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Open the website
url = 'https://www.statista.com/statistics/193534/franchise-value-of-national-football-league-teams-in-2010/'
driver.get(url)

In [10]:
# Click "Only necessary" for cookie options
# <button id="onetrust-reject-all-handler">Only necessary</button>
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[text()="Only necessary"]'))
)
button.click()


<button aria-label="Settings" aria-haspopup="true" data-gtm="statisticGraph__icon--settings" type="button" class="toolbar__item panelCard padding-all-10 flex-s--grow toolbar__item--active" tabindex="0"><svg aria-hidden="true" height="12" width="18" class="toolbar__icon uiIcon--default"><use href="#iconAdjustmentFill"></use></svg></button>



In [14]:
# Select "adjust statistic presentation" button (resembles normal "settings" button)
# <svg aria-hidden="true" height="12" width="18" class="toolbar__icon uiIcon--default"><use href="#iconAdjustmentFill"></use></svg>
adjust_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
adjust_button.click()

In [15]:
# Select table format button under chart type 
chart_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Settings"]'))
)
chart_button.click()

In [18]:
# Close Chrome driver
driver.quit()